In [87]:
import numpy as np
import igl
import meshplot as mp

In [88]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]

    delta = mmax-mmin

    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)

    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)

    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert (index == V.shape[0])

    tets = np.array([
        [0, 1, 3, 4],
        [5, 2, 6, 7],
        [4, 1, 5, 3],
        [4, 3, 7, 5],
        [3, 1, 5, 2],
        [2, 3, 7, 5]
    ])

    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]

                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :] = tmp
                    index += 1

    assert (index == T.shape[0])

    V += mmin
    return V, T

# Reading point cloud

In [89]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
print(pi)
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 5})

[[  0.    0.    0. ]
 [ -0.1  -6.    1. ]
 [ -4.4  -3.4  -1. ]
 ...
 [ 26.4 -37.2 -96.2]
 [ 26.2 -44.4 -96.1]
 [ 24.8 -50.2 -96.4]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Setting Up Constraint

In [90]:
# Compute Constraint Points

#This is the function finding closest points
def find_closest_point(point,points):
    distances=np.linalg.norm(points-point,axis=1)
    return np.argmin(distances)

def setup_constraints(pi,ni,eps,bbox_diag):
    #Fix an inital eps value
    eps=1

    # Initialize vectors for p, p+, p-, and their corresponding f values
    p = np.empty((0, 3), dtype=np.float64)
    f = np.empty((0,), dtype=np.float64)

    # We first find a global eps
    for i,point in enumerate(pi):

        # First compute p+
        p_plus=point+eps*ni[i]
        while find_closest_point(p_plus, pi) != i:
            eps /= 2

        # Then compute  p-
        p_minus=point-eps * ni[i]
        while find_closest_point(p_minus, pi) != i:
            eps /= 2
        
        # After that we put things together
    for i,point in enumerate(pi):
        # First compute 2 points
        p_plus=point+eps*ni[i]
        p_minus=point-eps*ni[i]

        #Then put things together
        p = np.append(p, [point], axis=0)  # Append original point
        p = np.append(p, [p_plus], axis=0)  # Append p_plus
        p = np.append(p, [p_minus], axis=0)  # Append p_minus

   

        f = np.append(f, 0.0)
        f = np.append(f, eps)
        f = np.append(f, -eps)

    return p,f

# MLS function

In [91]:
# Parameters
bbox_min = np.array([-100., -100., -100.])
bbox_max = np.array([100., 100., 100.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

eps=1
p,f = setup_constraints(pi, ni, eps, bbox_diag)

print(p)
print(f)

# Plotting
# Points on the surface will be blue (f == 0)
p_surface = p[f == 0.0]
# Points outside will be red (f > 0)
p_outside = p[f > 0.0]
# Points inside will be green (f < 0)
p_inside = p[f < 0.0]


# Correcting the plot command

plot = mp.plot(p_surface, shading={"point_size": 5, "point_color": "blue"})

# Plot points outside in red
plot.add_points(p_outside, shading={"point_size": 5, "point_color": "red"})

# Plot points inside in green
plot.add_points(p_inside, shading={"point_size": 5, "point_color": "green"})

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.85047873e-01  6.82929244e-02  9.80353692e-01]
 [ 1.85047873e-01 -6.82929244e-02 -9.80353692e-01]
 ...
 [ 2.48000000e+01 -5.02000000e+01 -9.64000000e+01]
 [ 2.56597080e+01 -5.06184969e+01 -9.66928524e+01]
 [ 2.39402920e+01 -4.97815031e+01 -9.61071476e+01]]
[ 0.  1. -1. ...  0.  1. -1.]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

2

In [92]:
# Generate grid n x n x n
n = 30
x, T = tet_grid((n, n, n), bbox_min - 0.05 *
                bbox_diag, bbox_max + 0.05 * bbox_diag)


def wendland_weight(r, h):
    return np.maximum(0, (1 - r / h)**4 * (4 * r / h + 1))


def polynomial_basis(xi, degree):
    if degree == 0:
        # Only the constant term
        return np.array([1])
    elif degree == 1:
        # Constant plus linear terms: 1, x, y, z
        return np.array([1, xi[0], xi[1], xi[2]])
    elif degree == 2:
        # Include quadratic terms: 1, x, y, z, x^2, y^2, z^2, xy, xz, yz
        return np.array([
            1, xi[0], xi[1], xi[2],
            xi[0]**2, xi[1]**2, xi[2]**2,
            xi[0]*xi[1], xi[0]*xi[2], xi[1]*xi[2]
        ])
    else:
        raise ValueError("Unsupported degree of polynomial basis functions.")


def evaluate_point(x, p, f, h, degree):
    # First let us build the base according to the degree
    # Determine the number of columns based on the degree
    if degree == 0:
        num_cols = 1
    elif degree == 1:
        num_cols = 4
    elif degree == 2:
        num_cols = 10
    else:
        raise ValueError("Unsupported degree of polynomial basis functions.")

    # Pre-allocate the base matrix with the appropriate shape
    # The number of rows is the length of x, and the number of columns is determined by the degree
    base_matrix = np.empty((len(f), num_cols), dtype=np.float64)

    # Fill the base matrix row by row
    for i, pi in enumerate(p):
        base_matrix[i, :] = polynomial_basis(pi, degree)

    # The next step is to build the weight matrix
    matrix_weight = np.zeros((len(f), len(f)), dtype=np.float64)
    fx = np.zeros((len(x), 1), dtype=np.float64)
    
    for i, xi in enumerate(x):
        # The next step is to build the weight matrix for each xi
        xi_basis=polynomial_basis(xi, degree)
        for j, point in enumerate(p):
            distance = np.linalg.norm(xi - point)
            matrix_weight[j, j] = wendland_weight(distance, h)
        leftmatrix=(base_matrix.T)@(matrix_weight)@(base_matrix)
        rightmatrix=(base_matrix.T)@(matrix_weight)@(f)
        coefficient = np.linalg.solve(leftmatrix, rightmatrix)
        fx[i]=xi_basis@coefficient
        print(fx[i])

    return fx

In [93]:
fx=evaluate_point(x,p,f,h=65,degree=0)

[0.00681231]
[0.00714458]
[0.00747734]
[0.00780127]
[0.00810443]
[0.00837254]
[0.00858987]
[0.00874058]
[0.0088106]
[0.0087896]
[0.00867266]
[0.00846109]
[0.00816238]
[0.00778909]
[0.0073571]
[0.00688371]
[0.00638586]
[0.00587885]
[0.00537551]
[0.00488594]
[0.00441753]
[0.00397526]
[0.00356211]
[0.00317943]
[0.00282738]
[0.00250526]
[0.00221178]
[0.00194528]
[0.00170389]
[0.00148568]
[0.00724162]
[0.0076261]
[0.00801503]
[0.00839759]
[0.0087595]
[0.00908323]
[0.00934907]
[0.00953696]
[0.00962897]
[0.00961202]
[0.00948016]
[0.00923566]
[0.00888872]
[0.00845579]
[0.0079572]
[0.00741446]
[0.00684808]
[0.00627599]
[0.0057128]
[0.00516952]
[0.00465386]
[0.00417068]
[0.00372253]
[0.00331021]
[0.00293324]
[0.00259031]
[0.00227952]
[0.00199869]
[0.00174548]
[0.00151754]
[0.00769342]
[0.0081362]
[0.00858844]
[0.00903768]
[0.00946684]
[0.00985451]
[0.01017609]
[0.01040626]
[0.01052231]
[0.0105079]
[0.01035609]
[0.01007069]
[0.00966563]
[0.00916246]
[0.00858697]
[0.00796566]
[0.00732303]
[0.00667

In [94]:
ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
plot=mp.plot(x, c=ind, shading={"point_size": 5, "width": 800, "height": 800})
plot.add_points(p_outside, shading={"point_size": 5, "point_color": "red"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

# Marching to extract surface

In [95]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(8.7033786…